**Microsoft Movie Studios**

Author: Mario Mocombe

**Overview**

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

**Business Problem**

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

Questions to consider:

* What are the business's plan points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?

**Data Understanding**

1) im.db.zip 

    A zipped SQLite database containing movie data from the website Internet Movie Data Base.

2) bom.movie_gross.csv.gz

    A compressed CSV file containing box office data from the website Box Office Mojo.

Questions to consider:

* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?


In [61]:
##Import Standard Packages
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# 1 IMDB DATABASE

In [62]:
### making a connection with the IMDB DATABASE using SQLite3
conn = sqlite3.connect('zippedData/im.db')

In [63]:
### setting up a cursor so I'm able to move through the database.
### Let's continue on and create a cursor.
##A cursor object is what can actually execute SQL commands. You create it by calling .cursor() on the connection.

cur = conn.cursor()
# (This is a special query for finding the table names. 
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")

In [64]:
## Use the fetchall method to find out the table names
## Fetch the result and store it in table_names
table_names = cur.fetchall()
table_names

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [65]:
pd.read_sql("SELECT * FROM movie_basics;", conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [66]:
pd.read_sql("SELECT * FROM movie_ratings ORDER BY movie_id;", conn)

,movie_id,averagerating,numvotes
0,tt0063540,7.0,77
1,tt0066787,7.2,43
2,tt0069049,6.9,4517
3,tt0069204,6.1,13
4,tt0100275,6.5,119
...,...,...,...
73851,tt9913084,6.2,6
73852,tt9914286,8.7,136
73853,tt9914642,8.5,8
73854,tt9914942,6.6,5


In [69]:
#########KEEP#########################

s = """
SELECT primary_title, start_year, runtime_minutes, genres, averagerating, numvotes 
FROM movie_basics
JOIN movie_ratings
USING(movie_id)
ORDER BY numvotes DESC;
"""
imdb = pd.read_sql(s, conn)
#######################################

In [70]:
imdb

,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
1,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
2,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334
3,Django Unchained,2012,165.0,"Drama,Western",8.4,1211405
4,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655
...,...,...,...,...,...,...
73851,Columbus,2018,85.0,Comedy,5.8,5
73852,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5
73853,July Kaatril,2019,NaN,Romance,9.0,5
73854,Swarm Season,2019,86.0,Documentary,6.2,5


In [42]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   primary_title    73856 non-null  object 
 1   start_year       73856 non-null  int64  
 2   runtime_minutes  66236 non-null  float64
 3   genres           73052 non-null  object 
 4   averagerating    73856 non-null  float64
 5   numvotes         73856 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 3.4+ MB


In [71]:
imdb.dropna(inplace=True)

In [72]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65720 entries, 0 to 73854
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   primary_title    65720 non-null  object 
 1   start_year       65720 non-null  int64  
 2   runtime_minutes  65720 non-null  float64
 3   genres           65720 non-null  object 
 4   averagerating    65720 non-null  float64
 5   numvotes         65720 non-null  int64  
dtypes: float64(2), int64(2), object(2)
memory usage: 3.5+ MB


In [74]:
imdb.head(20)

,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
1,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
2,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334
3,Django Unchained,2012,165.0,"Drama,Western",8.4,1211405
4,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655
5,The Wolf of Wall Street,2013,180.0,"Biography,Crime,Drama",8.2,1035358
6,Shutter Island,2010,138.0,"Mystery,Thriller",8.1,1005960
7,Guardians of the Galaxy,2014,121.0,"Action,Adventure,Comedy",8.1,948394
8,Deadpool,2016,108.0,"Action,Adventure,Comedy",8.0,820847
9,The Hunger Games,2012,142.0,"Action,Adventure,Sci-Fi",7.2,795227


In [73]:
imdb.describe()

,start_year,runtime_minutes,averagerating,numvotes
count,65720.000000,65720.000000,65720.000000,6.572000e+04
mean,2014.258065,94.732273,6.320902,3.954674e+03
std,2.600143,209.377017,1.458878,3.208823e+04
min,2010.000000,3.000000,1.000000,5.000000e+00
25%,2012.000000,81.000000,5.500000,1.600000e+01
50%,2014.000000,91.000000,6.500000,6.200000e+01
75%,2016.000000,104.000000,7.300000,3.520000e+02
max,2019.000000,51420.000000,10.000000,1.841066e+06


In [87]:
#Add > 2500 votes as a condition?

imdb.tail(64000)

,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes
1720,Fading Gigolo,2013,90.0,Comedy,6.2,22473
1721,The Guernsey Literary and Potato Peel Pie Society,2018,124.0,"Drama,History,Romance",7.4,22443
1722,The Silence,2019,90.0,"Horror,Thriller",5.2,22399
1723,Better Watch Out,2016,89.0,"Comedy,Crime,Horror",6.5,22367
1724,Paranormal Activity: The Ghost Dimension,2015,88.0,"Horror,Mystery,Thriller",4.6,22361
...,...,...,...,...,...,...
73848,The Winter Garden's Tale,2018,75.0,"Documentary,Drama",7.6,5
73850,The Projectionist,2019,81.0,Documentary,7.0,5
73851,Columbus,2018,85.0,Comedy,5.8,5
73852,BADMEN with a good behavior,2018,87.0,"Comedy,Horror",9.2,5


In [76]:
imdb.value_counts()

primary_title                              start_year  runtime_minutes  genres                averagerating  numvotes
Šiška Deluxe                               2015        108.0            Comedy,Drama          6.3            384         1
Goodbye to All That                        2014        87.0             Comedy,Drama,Romance  5.2            2141        1
Grace                                      2014        95.0             Drama                 6.0            176         1
                                           2011        98.0             Crime,Drama,Horror    6.5            19          1
Grabbers                                   2012        94.0             Comedy,Horror,Sci-Fi  6.3            15727       1
                                                                                                                        ..
Revelation: Dawn of Global Government      2016        106.0            Documentary           6.8            65          1
Revelation Trail     

In [86]:
imdb.duplicated().value_counts()

False    65720
dtype: int64

In [231]:
####SELECT FROM WHERE
    ###Genre LIKE '%Action%'

In [241]:
###DEL combined_df = pd.concat([x, y], axis=1, join='inner')

In [242]:
###DEL combined_df

In [243]:
###  combined_df

In [49]:
##TEST TEST#########---HIGHEST GROSSING ACTOR/ACTRESS---#####################

q = """
SELECT primary_title, runtime_minutes, genres, category, primary_name, averagerating, numvotes 
FROM principals
JOIN movie_ratings
USING (movie_id)
JOIN movie_basics
USING (movie_id)
JOIN persons
USING (person_id)
WHERE category = "actress"
OR category = "actor"
ORDER BY numvotes DESC;
"""
imdb2 = pd.read_sql(q, conn)
#######################################

In [50]:
imdb2.head(60)

,primary_title,runtime_minutes,genres,category,primary_name,averagerating,numvotes
0,Inception,148.0,"Action,Adventure,Sci-Fi",actor,Leonardo DiCaprio,8.8,1841066
1,Inception,148.0,"Action,Adventure,Sci-Fi",actor,Joseph Gordon-Levitt,8.8,1841066
2,Inception,148.0,"Action,Adventure,Sci-Fi",actress,Ellen Page,8.8,1841066
3,Inception,148.0,"Action,Adventure,Sci-Fi",actor,Ken Watanabe,8.8,1841066
4,The Dark Knight Rises,164.0,"Action,Thriller",actor,Christian Bale,8.4,1387769
5,The Dark Knight Rises,164.0,"Action,Thriller",actor,Tom Hardy,8.4,1387769
6,The Dark Knight Rises,164.0,"Action,Thriller",actress,Anne Hathaway,8.4,1387769
7,The Dark Knight Rises,164.0,"Action,Thriller",actor,Gary Oldman,8.4,1387769
8,Interstellar,169.0,"Adventure,Drama,Sci-Fi",actor,Matthew McConaughey,8.6,1299334
9,Interstellar,169.0,"Adventure,Drama,Sci-Fi",actress,Anne Hathaway,8.6,1299334


In [52]:
####TEST####
imdb2['primary_name'].value_counts()

Eric Roberts         122
Prakash Raj           74
Brahmanandam          74
Tom Sizemore          61
Michael Madsen        59
                    ... 
Juan Ferrara           1
Anthony Brogan         1
Gayla Goehl            1
Carolina Raspanti      1
Melody Tang            1
Name: primary_name, Length: 142527, dtype: int64

In [53]:
####TEST####
imdb2['genres'].value_counts().head(20)

Drama                   48535
Comedy                  24363
Horror                  12110
Comedy,Drama            11008
Thriller                 6608
Drama,Romance            6301
Documentary              6247
Comedy,Romance           5339
Comedy,Drama,Romance     5056
Action                   4502
Horror,Thriller          4225
Drama,Thriller           4140
Romance                  3094
Comedy,Horror            2551
Action,Crime,Drama       2399
Crime,Drama,Thriller     2093
Crime,Drama              2079
Family                   2015
Drama,Family             1980
Action,Drama             1738
Name: genres, dtype: int64

In [ ]:
conn.close()

# 2 BOX OFFICE MOJO

In [165]:
##Loading up the first dataframe, BOX OFFICE MOJO, with Pandas.Importing Data
# Import the file and print the first 5 rows
df = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
df

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [166]:
df.info()
### CHANGE FOREIGN GROSS TO FLOAT

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [192]:
## df['foreign_gross'].astype(float)
##Gives Error could not convert string to float: '1,131.6'

In [198]:
### sorting the values by domestic gross, we see that the foreign gross is off for 3 of the top results and for Furious 7.
df.sort_values(by=['domestic_gross'],ascending=False)

,title,studio,domestic_gross,foreign_gross,year
1872,Star Wars: The Force Awakens,BV,936700000.0,"1,131.6",2015
3080,Black Panther,BV,700100000.0,646900000,2018
3079,Avengers: Infinity War,BV,678800000.0,"1,369.5",2018
1873,Jurassic World,Uni.,652300000.0,"1,019.4",2015
727,Marvel's The Avengers,BV,623400000.0,895500000,2012
...,...,...,...,...,...
1975,Surprise - Journey To The West,AR,NaN,49600000,2015
2392,Finding Mr. Right 2,CL,NaN,114700000,2016
2468,Solace,LGP,NaN,22400000,2016
2595,Viral,W/Dim.,NaN,552000,2016


In [199]:
### sorting the values by foreign gross, we see that the top 5 results are wildly popular franchises with a little over $1k gross.
df.sort_values(by=['foreign_gross'])

,title,studio,domestic_gross,foreign_gross,year
2760,The Fate of the Furious,Uni.,226000000.0,"1,010.0",2017
1873,Jurassic World,Uni.,652300000.0,"1,019.4",2015
1872,Star Wars: The Force Awakens,BV,936700000.0,"1,131.6",2015
1874,Furious 7,Uni.,353000000.0,"1,163.0",2015
3079,Avengers: Infinity War,BV,678800000.0,"1,369.5",2018
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [200]:
## REPLACE the erroneous values with more realistic ones.  The five entries were most likely meant to contain billions in gross.
df['foreign_gross'] = df['foreign_gross'].replace(['1,010.0','1,019.4','1,131.6', '1,163.0','1,369.5'], ['1010000000', '1019000000', '1131000000', '1163000000', '1369000000'])

In [204]:
#check to see if the values changed
df.sort_values(by=['domestic_gross'],ascending=False)

,title,studio,domestic_gross,foreign_gross,year
1872,Star Wars: The Force Awakens,BV,936700000.0,1131000000,2015
3080,Black Panther,BV,700100000.0,646900000,2018
3079,Avengers: Infinity War,BV,678800000.0,1369000000,2018
1873,Jurassic World,Uni.,652300000.0,1019000000,2015
727,Marvel's The Avengers,BV,623400000.0,895500000,2012
...,...,...,...,...,...
1975,Surprise - Journey To The West,AR,NaN,49600000,2015
2392,Finding Mr. Right 2,CL,NaN,114700000,2016
2468,Solace,LGP,NaN,22400000,2016
2595,Viral,W/Dim.,NaN,552000,2016


In [212]:
df['foreign_gross'].astype(float)

0       652000000.0
1       691300000.0
2       664300000.0
3       535700000.0
4       513900000.0
           ...     
3382            NaN
3383            NaN
3384            NaN
3385            NaN
3386            NaN
Name: foreign_gross, Length: 3387, dtype: float64

In [210]:
df['foreign_gross'].dtypes

dtype('O')

In [213]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [181]:
df.groupby('domestic_gross')

df.groupby(df['domestic_gross'])

In [183]:
df.groupby('domestic_gross').mean()

,year
domestic_gross,
100.0,2013.000000
300.0,2015.666667
400.0,2014.500000
500.0,2017.000000
600.0,2012.000000
...,...
623400000.0,2012.000000
652300000.0,2015.000000
678800000.0,2018.000000


In [ ]:
##DROP MISSING VALUES FOR FOREIGN GROSS, DO new df.tail

In [151]:
df["foreign_gross"].tail(60)

3327      NaN
3328      NaN
3329      NaN
3330      NaN
3331      NaN
3332      NaN
3333      NaN
3334      NaN
3335      NaN
3336      NaN
3337      NaN
3338      NaN
3339      NaN
3340      NaN
3341      NaN
3342     5200
3343      NaN
3344      NaN
3345      NaN
3346      NaN
3347      NaN
3348      NaN
3349      NaN
3350      NaN
3351      NaN
3352      NaN
3353    30000
3354      NaN
3355      NaN
3356      NaN
3357      NaN
3358      NaN
3359      NaN
3360      NaN
3361      NaN
3362      NaN
3363      NaN
3364      NaN
3365      NaN
3366      NaN
3367      NaN
3368      NaN
3369      NaN
3370      NaN
3371      NaN
3372      NaN
3373      NaN
3374      NaN
3375      NaN
3376      NaN
3377      NaN
3378      NaN
3379      NaN
3380      NaN
3381      NaN
3382      NaN
3383      NaN
3384      NaN
3385      NaN
3386      NaN
Name: foreign_gross, dtype: object

In [ ]:
#df['total gross'] = df['domestic_gross'] + df['foreign_gross']

In [111]:
df.index

RangeIndex(start=0, stop=3387, step=1)

In [126]:
df.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [47]:
df.columns

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')

In [48]:
df.isna().sum()

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

In [49]:
df.dtypes

###change foreign_gross into float

title              object
studio             object
domestic_gross    float64
foreign_gross      object
year                int64
dtype: object

In [113]:
df.sorted("foreign_gross")

AttributeError: 'DataFrame' object has no attribute 'sort'

In [50]:
df.head(20)

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
5,The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
6,Iron Man 2,Par.,312400000.0,311500000,2010
7,Tangled,BV,200800000.0,391000000,2010
8,Despicable Me,Uni.,251500000.0,291600000,2010
9,How to Train Your Dragon,P/DW,217600000.0,277300000,2010


In [132]:
df["studio"].value_counts().head(50)

IFC         166
Uni.        147
WB          140
Fox         136
Magn.       136
SPC         123
Sony        110
BV          106
LGF         103
Par.        101
Eros         89
Wein.        77
CL           74
Strand       68
FoxS         67
RAtt.        66
KL           62
Focus        60
WGUSA        58
CJ           56
MBox         54
UTV          50
A24          49
WB (NL)      45
FM           42
LG/S         41
Cohen        40
ORF          37
SGem         35
Rela.        35
FIP          26
STX          24
Gold.        24
GK           24
Osci.        23
RTWC         23
TriS         23
BST          22
MNE          22
EOne         21
Drft.        20
Distrib.     20
Relbig.      18
Anch.        18
CBS          17
Zeit.        16
BG           16
Sum.         15
FD           14
W/Dim.       14
Name: studio, dtype: int64

In [52]:
df.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [53]:
df.shape

(3387, 5)

# 3 THE NUMBERS

In [54]:
##Loading up the second dataframe, THE NUMBERS, with Pandas.
df2 = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")
df2

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [55]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [56]:
df2.shape

(5782, 6)

In [ ]:
##### RELEASE DATES / WORLDWIDEGROSS

In [57]:
df2.index

RangeIndex(start=0, stop=5782, step=1)

In [58]:
df2.columns

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [59]:
df2.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [60]:
df2.dtypes

id                    int64
release_date         object
movie                object
production_budget    object
domestic_gross       object
worldwide_gross      object
dtype: object

In [61]:
df2.value_counts()

id   release_date  movie                     production_budget  domestic_gross  worldwide_gross
100  Sep 2, 2005   The Transporter 2         $32,000,000        $43,095,856     $88,978,458        1
34   Apr 30, 2010  Housefull                 $10,100,000        $1,183,658      $18,726,300        1
     Apr 5, 2019   The Best of Enemies       $10,000,000        $10,205,616     $10,205,616        1
     Aug 13, 2010  The Expendables           $82,000,000        $103,068,524    $268,268,174       1
     Aug 25, 2017  Birth of the Dragon       $31,000,000        $6,901,965      $7,220,490         1
                                                                                                  ..
67   Jun 15, 2005  Batman Begins             $150,000,000       $205,343,774    $359,142,722       1
     Jun 19, 1987  The Brave Little Toaster  $2,300,000         $0              $0                 1
     Jun 3, 1988   Big                       $18,000,000        $114,968,774    $151,668,774    

# 4 THE MOVIE DATABASE

In [ ]:
### DROP DUPLICATES

In [239]:
##Loading up the third dataframe, THE MOVIE DATABASE, with Pandas.
##Specify the argument index_col=0 in order to avoid creating an extra Id column

df3 = pd.read_csv("zippedData/tmdb.movies.csv.gz", index_col=0)
df3

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...,...,...
26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [63]:
df3.describe()

,Unnamed: 0,id,popularity,vote_average,vote_count
count,26517.00000,26517.000000,26517.000000,26517.000000,26517.000000
mean,13258.00000,295050.153260,3.130912,5.991281,194.224837
std,7654.94288,153661.615648,4.355229,1.852946,960.961095
min,0.00000,27.000000,0.600000,0.000000,1.000000
25%,6629.00000,157851.000000,0.600000,5.000000,2.000000
50%,13258.00000,309581.000000,1.374000,6.000000,5.000000
75%,19887.00000,419542.000000,3.694000,7.000000,28.000000
max,26516.00000,608444.000000,80.773000,10.000000,22186.000000


In [64]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [65]:
df3.columns

Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')

In [66]:
df3.shape

(26517, 10)

In [67]:
df3.index

RangeIndex(start=0, stop=26517, step=1)

In [68]:
df3.isna().sum()

Unnamed: 0           0
genre_ids            0
id                   0
original_language    0
original_title       0
popularity           0
release_date         0
title                0
vote_average         0
vote_count           0
dtype: int64

In [69]:
df3.dtypes

Unnamed: 0             int64
genre_ids             object
id                     int64
original_language     object
original_title        object
popularity           float64
release_date          object
title                 object
vote_average         float64
vote_count             int64
dtype: object

In [70]:
df3.value_counts()

Unnamed: 0  genre_ids            id      original_language  original_title                                popularity  release_date  title                                         vote_average  vote_count
26516       [53, 27]             309885  en                 The Church                                    0.600       2018-10-05    The Church                                    0.0           1             1
8841        [878, 35]            167948  en                 Space Milkshake                               2.219       2013-02-15    Space Milkshake                               6.1           18            1
8831        [18, 36, 10749]      193006  en                 Mary, Queen of Scots                          2.242       2013-09-08    Mary, Queen of Scots                          4.7           16            1
8832        [35]                 179690  ru                 Что творят мужчины!                           2.238       2013-03-03    What Men Do!                             

In [203]:
import csv

# 5A ROTTEN TOMATOES REVIEWS

In [232]:
## Opened up the .gz file with WinZip.  Inside was the .tsv file.
## Using pd.read_csv to open .tsv file gave UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf1 in position 2: invalid continuation byte
## Fixed this error by using 'latin-1' encoding

df5 = pd.read_csv("zippedData/rt.reviews.tsv", sep='\t', encoding='latin-1')
df5

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


In [236]:
df5['fresh'].value_counts()

fresh     33035
rotten    21397
Name: fresh, dtype: int64

In [210]:
with open("zippedData/rt.reviews.tsv") as f:
    reader = csv.reader(f)
    # Printing only the header and first 5 rows of data
    for _ in range(100):
        print(next(reader))

['id\treview\trating\tfresh\tcritic\ttop_critic\tpublisher\tdate']
['3\tA distinctly gallows take on contemporary financial mores', " as one absurdly rich man's limo ride across town for a haircut functions as a state-of-the-nation discourse. \t3/5\tfresh\tPJ Nabarro\t0\tPatrick Nabarro\tNovember 10", ' 2018']
["3\tIt's an allegory in search of a meaning that never arrives...It's just old-fashioned bad storytelling.\t\trotten\tAnnalee Newitz\t0\tio9.com\tMay 23", ' 2018']
['3\t... life lived in a bubble in financial dealings and digital communications and brief face-to-face conversations and sexual intermissions in a space shuttle of a limousine creeping through the gridlock of an anonymous New York City.\t\tfresh\tSean Axmaker\t0\tStream on Demand\tJanuary 4', ' 2018']
['3\t"Continuing along a line introduced in last year\'s ""A Dangerous Method""', ' David Cronenberg pushes his cinema towards a talky abstraction in his uncanny', ' perversely funny and frighteningly insular adaptation

In [ ]:
###usa_2016_gold_medals = []

##for row in olympics_data:
##    if row["Medal"] == "G" and row["Nationality"] == "USA" and row["Year"] == "2016":
##        usa_2016_gold_medals.append({"Event": row["Event"], "Name": row["Name"]})
        
## usa_2016_gold_medals

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"
...,...,...,...,...,...,...,...,...
54427,2000,The real charm of this trifle is the deadpan c...,NaN,fresh,Laura Sinagra,1,Village Voice,"September 24, 2002"
54428,2000,NaN,1/5,rotten,Michael Szymanski,0,Zap2it.com,"September 21, 2005"
54429,2000,NaN,2/5,rotten,Emanuel Levy,0,EmanuelLevy.Com,"July 17, 2005"
54430,2000,NaN,2.5/5,rotten,Christopher Null,0,Filmcritic.com,"September 7, 2003"


# 5B ROTTEN TOMATOES MOVIE INFO

In [ ]:
#Correlation between $Box Office$ 
# Average run-time for top X amount of films
# Ratings for top X amount of films
#
#

In [168]:
df6 = pd.read_csv("zippedData/rt.movie_info.tsv", sep='\t')
df6

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1555,1996,Forget terrorists or hijackers -- there's a ha...,R,Action and Adventure|Horror|Mystery and Suspense,NaN,NaN,"Aug 18, 2006","Jan 2, 2007",$,"33,886,034",106 minutes,New Line Cinema
1556,1997,The popular Saturday Night Live sketch was exp...,PG,Comedy|Science Fiction and Fantasy,Steve Barron,Terry Turner|Tom Davis|Dan Aykroyd|Bonnie Turner,"Jul 23, 1993","Apr 17, 2001",NaN,NaN,88 minutes,Paramount Vantage
1557,1998,"Based on a novel by Richard Powell, when the l...",G,Classics|Comedy|Drama|Musical and Performing Arts,Gordon Douglas,NaN,"Jan 1, 1962","May 11, 2004",NaN,NaN,111 minutes,NaN
1558,1999,The Sandlot is a coming-of-age story about a g...,PG,Comedy|Drama|Kids and Family|Sports and Fitness,David Mickey Evans,David Mickey Evans|Robert Gunter,"Apr 1, 1993","Jan 29, 2002",NaN,NaN,101 minutes,NaN


In [178]:
df6.dtypes

id               int64
synopsis        object
rating          object
genre           object
director        object
writer          object
theater_date    object
dvd_date        object
currency        object
box_office      object
runtime         object
studio          object
dtype: object

In [181]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [182]:
df6.columns 

Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

In [183]:
df6.isna().sum()

id                 0
synopsis          62
rating             3
genre              8
director         199
writer           449
theater_date     359
dvd_date         359
currency        1220
box_office      1220
runtime           30
studio          1066
dtype: int64

In [185]:
df6.value_counts

<bound method DataFrame.value_counts of         id                                           synopsis rating  \
0        1  This gritty, fast-paced, and innovative police...      R   
1        3  New York City, not-too-distant-future: Eric Pa...      R   
2        5  Illeana Douglas delivers a superb performance ...      R   
3        6  Michael Douglas runs afoul of a treacherous su...      R   
4        7                                                NaN     NR   
...    ...                                                ...    ...   
1555  1996  Forget terrorists or hijackers -- there's a ha...      R   
1556  1997  The popular Saturday Night Live sketch was exp...     PG   
1557  1998  Based on a novel by Richard Powell, when the l...      G   
1558  1999  The Sandlot is a coming-of-age story about a g...     PG   
1559  2000  Suspended from the force, Paris cop Hubert is ...      R   

                                                  genre            director  \
0               

In [186]:
df6.describe()

,id
count,1560.000000
mean,1007.303846
std,579.164527
min,1.000000
25%,504.750000
50%,1007.500000
75%,1503.250000
max,2000.000000


In [187]:
df6.shape

(1560, 12)

In [189]:
df6.tail(20)

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
1540,1979,"In this taut crime drama, a radio broadcaster ...",G,Classics|Drama|Mystery and Suspense,Michael Curtiz,Ranald MacDougall|Bess Meredyth,NaN,NaN,NaN,NaN,103 minutes,NaN
1541,1980,A band of renegades on the run in outer space ...,PG-13,Action and Adventure|Science Fiction and Fantasy,Joss Whedon,Joss Whedon,"Sep 30, 2005","Dec 20, 2005",$,"25,335,935",119 minutes,Universal Pictures
1542,1981,"Money, Fame and the Knowledge of English. In I...",NR,Comedy|Drama,Gauri Shinde,Gauri Shinde,"Oct 5, 2012","Nov 20, 2012",$,"1,416,189",129 minutes,Eros Entertainment
1543,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1544,1983,This animated children's film spoofs the Sherl...,G,Action and Adventure|Animation|Kids and Family,Burny Mattinson|Ron Clements|John Musker|Dave ...,NaN,"Jul 2, 1986","Jul 23, 2002",NaN,NaN,74 minutes,NaN
1545,1985,A woman who joins the undead against her will ...,R,Horror|Mystery and Suspense,Sebastian Gutierrez,Sebastian Gutierrez,"Jun 1, 2007","Oct 9, 2007",$,"59,371",98 minutes,IDP Distribution
1546,1986,Aki Kaurismaki's The Man Without a Past opens ...,PG,Art House and International|Comedy|Drama,NaN,NaN,"Aug 30, 2002","Oct 7, 2003",$,"794,306",97 minutes,NaN
1547,1987,Patrick Dempsey stars in this biography of Son...,PG-13,Comedy,Phil Alden Robinson,Phil Alden Robinson,NaN,NaN,NaN,NaN,98 minutes,NaN
1548,1988,When the money-hungry Duke Stuyvesant (Sterlin...,R,Comedy,Les Rose,NaN,NaN,NaN,NaN,NaN,94 minutes,NaN
1549,1989,Hungarian Rhapsody (Magyar Rapszodia) is the f...,NR,Art House and International|Drama,NaN,NaN,NaN,NaN,NaN,NaN,101 minutes,NaN


**Conclusions**

Provide your conclusions about the work you've done, including any limitations or next steps.


Questions to consider:

* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?